# Setup

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn import preprocessing
from pylab import rcParams

In [2]:
users_df = pd.read_excel(r'./data/userdata.xlsx')
network_df = pd.read_excel(r'./data/network2.xlsx')

# If index column remains
users_df.drop(users_df.columns[0], axis=1, inplace=True)
network_df.drop(network_df.columns[0], axis=1, inplace=True)

In [3]:
users_df.head(10)

,user,listed,followers
0,cloth_couture,0,1
1,Britany82060809,0,97
2,fifakitcreator,3,1476
3,DiscountTreasu1,0,133
4,Scarlet2Gray,0,42
5,salepricejo,0,12
6,IsabelBarreir12,0,9
7,MeSnooty,1,230
8,CFlirtwear,0,0
9,DreamsofaOwl,6,301


In [4]:
network_df.head(10)

,Source,Sourceid,Target,Targetid,type_of_content
0,mrsmcpuffin,60245403,FAMU_MBB,2726221854,Retweet
1,mrsmcpuffin,60245403,KingJames,23083404,Retweet
2,somenailstrips,1094314346593861633,WandaCo87259801,1117986510551302144,Retweet
3,somenailstrips,1094314346593861633,Poshmarkapp,357211620,Retweet
4,ja_corey14,3774742874,originelllly,1663859550,Retweet
5,BIGGMIKE904,66279549,HCWillieSimmons,29780461,Retweet
6,BIGGMIKE904,66279549,FAMU_MBB,2726221854,Retweet
7,BIGGMIKE904,66279549,KingJames,23083404,Retweet
8,_kingfree,1541895800,FAMU_MBB,2726221854,Retweet
9,_kingfree,1541895800,KingJames,23083404,Retweet


In [5]:
network_df.rename(columns={'Source': 'source', 'Sourceid': 'source_id', 'Target': 'target',
                          'Targetid': 'target_id'}, inplace=True)

In [6]:
network_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   source           6012 non-null   object
 1   source_id        6012 non-null   int64 
 2   target           6012 non-null   object
 3   target_id        6012 non-null   int64 
 4   type_of_content  6012 non-null   object
dtypes: int64(2), object(3)
memory usage: 235.0+ KB


In [7]:
network_df['type_of_content'].value_counts()

Retweet    2281
Mention    1902
Tweet      1694
Reply       135
Name: type_of_content, dtype: int64

In [8]:
network_df['type_of_content'].replace(['Retweet', 'Mention', 'Reply'], 'Non-Tweet', inplace=True)

In [9]:
network_df['type_of_content'].value_counts()

Non-Tweet    4318
Tweet        1694
Name: type_of_content, dtype: int64

In [10]:
network_df.head(10)

,source,source_id,target,target_id,type_of_content
0,mrsmcpuffin,60245403,FAMU_MBB,2726221854,Non-Tweet
1,mrsmcpuffin,60245403,KingJames,23083404,Non-Tweet
2,somenailstrips,1094314346593861633,WandaCo87259801,1117986510551302144,Non-Tweet
3,somenailstrips,1094314346593861633,Poshmarkapp,357211620,Non-Tweet
4,ja_corey14,3774742874,originelllly,1663859550,Non-Tweet
5,BIGGMIKE904,66279549,HCWillieSimmons,29780461,Non-Tweet
6,BIGGMIKE904,66279549,FAMU_MBB,2726221854,Non-Tweet
7,BIGGMIKE904,66279549,KingJames,23083404,Non-Tweet
8,_kingfree,1541895800,FAMU_MBB,2726221854,Non-Tweet
9,_kingfree,1541895800,KingJames,23083404,Non-Tweet


In [11]:
non_tweet_df = network_df[network_df['type_of_content'] == 'Non-Tweet']
tweet_df = network_df[network_df['type_of_content'] == 'Tweet']

In [12]:
# Calculate number of tweets
# user_df['tweets'] = 
users_df['tweets'] = pd.to_numeric(tweet_df.groupby('source')['type_of_content'].transform('count'))

In [13]:
# Calculate non-tweets sent and received
users_df['non_tweets_sent'] = pd.to_numeric(non_tweet_df.groupby('source')['type_of_content'].transform('count'))
users_df['non_tweets_received'] = pd.to_numeric(non_tweet_df.groupby('target')['type_of_content'].transform('count'))

# Graph

In [14]:
users_df.fillna(0, inplace=True)
users_df

,user,listed,followers,tweets,non_tweets_sent,non_tweets_received
0,cloth_couture,0,1,0.0,2.0,110.0
1,Britany82060809,0,97,0.0,2.0,124.0
2,fifakitcreator,3,1476,0.0,2.0,4.0
3,DiscountTreasu1,0,133,0.0,2.0,1207.0
4,Scarlet2Gray,0,42,0.0,1.0,1.0
...,...,...,...,...,...,...
3972,PickerTesla,0,29,0.0,3.0,1.0
3973,Cop_Market,1,1327,0.0,1.0,1207.0
3974,Gluxia1,0,4,0.0,2.0,1207.0
3975,rgikpttn,11,56,0.0,2.0,3.0


In [15]:
G = nx.DiGraph()

edges = list()
for (a,b) in zip(non_tweet_df['source'], non_tweet_df['target']):
    edges.append((a,b))

G.add_edges_from(edges)
G

In [16]:
# Plot graph
'''
rcParams['figure.figsize'] = 14, 10
pos = nx.spring_layout(G, scale=20, k=3/np.sqrt(G.order()))
nx.draw(G, pos, node_color='lightblue')
'''

"\nrcParams['figure.figsize'] = 14, 10\npos = nx.spring_layout(G, scale=20, k=3/np.sqrt(G.order()))\nnx.draw(G, pos, node_color='lightblue')\n"

In [17]:
degree_centrality=pd.DataFrame.from_dict(nx.degree_centrality(G), orient='index').reset_index()
degree_centrality

,index,0
0,mrsmcpuffin,0.000576
1,FAMU_MBB,0.031970
2,KingJames,0.034562
3,somenailstrips,0.000576
4,WandaCo87259801,0.001440
...,...,...
3468,SteakxEggs,0.000288
3469,KaysUniverse11,0.000288
3470,Gluxia1,0.000288
3471,KingSize199619,0.000288


In [18]:
closeness_centrality=pd.DataFrame.from_dict(nx.closeness_centrality(G), orient='index').reset_index()
closeness_centrality

,index,0
0,mrsmcpuffin,0.000000
1,FAMU_MBB,0.031685
2,KingJames,0.034562
3,somenailstrips,0.000000
4,WandaCo87259801,0.001152
...,...,...
3468,SteakxEggs,0.000288
3469,KaysUniverse11,0.000288
3470,Gluxia1,0.000000
3471,KingSize199619,0.000288


In [19]:
betweenness_centrality=pd.DataFrame.from_dict(nx.betweenness_centrality(G), orient='index').reset_index()
betweenness_centrality

,index,0
0,mrsmcpuffin,0.0
1,FAMU_MBB,0.0
2,KingJames,0.0
3,somenailstrips,0.0
4,WandaCo87259801,0.0
...,...,...
3468,SteakxEggs,0.0
3469,KaysUniverse11,0.0
3470,Gluxia1,0.0
3471,KingSize199619,0.0


In [20]:
network_measures = degree_centrality.merge(closeness_centrality, on='index').merge(betweenness_centrality, on='index')
network_measures

,index,0_x,0_y,0
0,mrsmcpuffin,0.000576,0.000000,0.0
1,FAMU_MBB,0.031970,0.031685,0.0
2,KingJames,0.034562,0.034562,0.0
3,somenailstrips,0.000576,0.000000,0.0
4,WandaCo87259801,0.001440,0.001152,0.0
...,...,...,...,...
3468,SteakxEggs,0.000288,0.000288,0.0
3469,KaysUniverse11,0.000288,0.000288,0.0
3470,Gluxia1,0.000288,0.000000,0.0
3471,KingSize199619,0.000288,0.000288,0.0


In [21]:
network_measures.columns = ['user', 'degree', 'closeness', 'betweenness']

In [22]:
network_measures = network_measures.merge(users_df, on='user')
network_measures

,user,degree,closeness,betweenness,listed,followers,tweets,non_tweets_sent,non_tweets_received
0,mrsmcpuffin,0.000576,0.000000,0.0,37,1626,0.0,2.0,124.0
1,FAMU_MBB,0.031970,0.031685,0.0,43,2744,0.0,1.0,144.0
2,KingJames,0.034562,0.034562,0.0,44542,49401456,0.0,2.0,110.0
3,somenailstrips,0.000576,0.000000,0.0,7,2506,27.0,0.0,0.0
4,WandaCo87259801,0.001440,0.001152,0.0,2,280,0.0,2.0,1207.0
...,...,...,...,...,...,...,...,...,...
3359,SteakxEggs,0.000288,0.000288,0.0,0,275,0.0,1.0,144.0
3360,KaysUniverse11,0.000288,0.000288,0.0,0,47,4.0,0.0,0.0
3361,Gluxia1,0.000288,0.000000,0.0,0,4,0.0,2.0,1207.0
3362,KingSize199619,0.000288,0.000288,0.0,2,1453,1.0,0.0,0.0


In [23]:
network_measures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3364 entries, 0 to 3363
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user                 3364 non-null   object 
 1   degree               3364 non-null   float64
 2   closeness            3364 non-null   float64
 3   betweenness          3364 non-null   float64
 4   listed               3364 non-null   int64  
 5   followers            3364 non-null   int64  
 6   tweets               3364 non-null   float64
 7   non_tweets_sent      3364 non-null   float64
 8   non_tweets_received  3364 non-null   float64
dtypes: float64(6), int64(2), object(1)
memory usage: 262.8+ KB


In [24]:
network_measures.set_index('user', inplace=True)

In [25]:
'''
d_network_measures = pd.DataFrame(columns=['A_user', 'B_user', 'dAB_degree', 'dAB_closeness', 'dAB_betweenness', 'dAB_listed', 
                                          'dAB_followers', 'dAB_tweets', 'dAB_non_tweets_sent', 'dAB_non_tweets_received'])
for user_a, row_a in network_measures.iterrows():
    for user_b, row_b in network_measures.iterrows():
        d_network_measures.append({
            'A_user': user_a,
            'B_user': user_b,
            'dAB_degree': row_a['degree'] - row_b['degree'],
            'dAB_closeness': row_a['closeness'] - row_b['closeness'],
            'dAB_betweenness': row_a['betweenness'] - row_b['betweenness'],
            'dAB_listed': row_a['listed'] - row_b['listed'],
            'dAB_followers': row_a['followers'] - row_b['followers'],
            'dAB_tweets': row_a['tweets'] - row_b['tweets'],
            'dAB_non_tweets_sent': row_a['non_tweets_sent'] - row_b['non_tweets_sent'],
            'dAB_non_tweets_received': row_a['non_tweets_received'] - row_b['non_tweets_received'],
        }, ignore_index=True)
d_network_measures
'''

"\nd_network_measures = pd.DataFrame(columns=['A_user', 'B_user', 'dAB_degree', 'dAB_closeness', 'dAB_betweenness', 'dAB_listed', \n                                          'dAB_followers', 'dAB_tweets', 'dAB_non_tweets_sent', 'dAB_non_tweets_received'])\nfor user_a, row_a in network_measures.iterrows():\n    for user_b, row_b in network_measures.iterrows():\n        d_network_measures.append({\n            'A_user': user_a,\n            'B_user': user_b,\n            'dAB_degree': row_a['degree'] - row_b['degree'],\n            'dAB_closeness': row_a['closeness'] - row_b['closeness'],\n            'dAB_betweenness': row_a['betweenness'] - row_b['betweenness'],\n            'dAB_listed': row_a['listed'] - row_b['listed'],\n            'dAB_followers': row_a['followers'] - row_b['followers'],\n            'dAB_tweets': row_a['tweets'] - row_b['tweets'],\n            'dAB_non_tweets_sent': row_a['non_tweets_sent'] - row_b['non_tweets_sent'],\n            'dAB_non_tweets_received': row_a

# Score 

In [26]:
feature_importance = pd.read_csv('.\data\\feature_importance.csv')
feature_importance = feature_importance.T
feature_importance.columns = feature_importance.iloc[0,:]
feature_importance.drop(feature_importance.index[0], inplace=True)
feature_importance

Unnamed: 0,dAB_follower,dAB_listed,dAB_tweets,dAB_degree,dAB_non_tweets_sent,dAB_non_tweets_received
Importance,0.09368,0.478216,0.077184,0.148541,0.07966,0.122718


In [27]:
feature_importance['dAB_follower'].values[0]

0.0936803743243217

In [28]:
network_measures['score'] = (
    network_measures['followers']*feature_importance['dAB_follower'].values +
    network_measures['listed']*feature_importance['dAB_listed'].values + 
    network_measures['tweets']*feature_importance['dAB_tweets'].values + 
    network_measures['degree']*feature_importance['dAB_degree'].values + 
    network_measures['non_tweets_sent']*feature_importance['dAB_non_tweets_sent'].values + 
    network_measures['non_tweets_received']*feature_importance['dAB_non_tweets_received'].values)                 

network_measures['score'] = network_measures['score'].astype(np.float32)
network_measures

,degree,closeness,betweenness,listed,followers,tweets,non_tweets_sent,non_tweets_received,score
user,,,,,,,,,
mrsmcpuffin,0.000576,0.000000,0.0,37,1626,0.0,2.0,124.0,1.853947e+02
FAMU_MBB,0.031970,0.031685,0.0,43,2744,0.0,1.0,144.0,2.953780e+02
KingJames,0.034562,0.034562,0.0,44542,49401456,0.0,2.0,110.0,4.649262e+06
somenailstrips,0.000576,0.000000,0.0,7,2506,27.0,0.0,0.0,2.401946e+02
WandaCo87259801,0.001440,0.001152,0.0,2,280,0.0,2.0,1207.0,1.754665e+02
...,...,...,...,...,...,...,...,...,...
SteakxEggs,0.000288,0.000288,0.0,0,275,0.0,1.0,144.0,4.351313e+01
KaysUniverse11,0.000288,0.000288,0.0,0,47,4.0,0.0,0.0,4.711758e+00
Gluxia1,0.000288,0.000000,0.0,0,4,0.0,2.0,1207.0,1.486541e+02


**Normalize the dataframe**

In [29]:
normalized_network_measures = (network_measures-network_measures.mean())/network_measures.std()
normalized_network_measures

,degree,closeness,betweenness,listed,followers,tweets,non_tweets_sent,non_tweets_received,score
user,,,,,,,,,
mrsmcpuffin,-0.011751,-0.059671,-0.068793,-0.082803,-0.066997,-0.301843,-0.081432,-0.258220,-0.067249
FAMU_MBB,6.022822,6.027018,-0.068793,-0.081094,-0.066550,-0.301843,-0.235158,-0.213846,-0.066783
KingJames,6.521090,6.579815,-0.068793,12.587767,19.667478,-0.301843,-0.081432,-0.289282,19.639791
somenailstrips,-0.011751,-0.059671,-0.068793,-0.091344,-0.066645,1.217122,-0.388883,-0.533340,-0.067017
WandaCo87259801,0.154339,0.161646,-0.068793,-0.092767,-0.067534,-0.301843,-0.081432,2.144640,-0.067291
...,...,...,...,...,...,...,...,...,...
SteakxEggs,-0.067114,-0.004342,-0.068793,-0.093336,-0.067536,-0.301843,-0.235158,-0.213846,-0.067851
KaysUniverse11,-0.067114,-0.004342,-0.068793,-0.093336,-0.067627,-0.076811,-0.388883,-0.533340,-0.068015
Gluxia1,-0.067114,-0.059671,-0.068793,-0.093336,-0.067645,-0.301843,-0.081432,2.144640,-0.067405


**Get top 100**

In [30]:
results = normalized_network_measures.nlargest(100, 'score', keep='all')
results

,degree,closeness,betweenness,listed,followers,tweets,non_tweets_sent,non_tweets_received,score
user,,,,,,,,,
Cristiano,-0.067114,-0.004342,-0.068793,23.742310,36.527049,1.160864,-0.388883,-0.533340,36.478227
YouTube,0.652606,0.714936,-0.068793,22.411054,29.124878,-0.301843,-0.081432,-0.289282,29.110755
KingJames,6.521090,6.579815,-0.068793,12.587767,19.667478,-0.301843,-0.081432,-0.289282,19.639791
instagram,-0.011751,0.064820,-0.068793,8.901188,13.806620,-0.301843,-0.081432,-0.289282,13.787621
kanyewest,-0.067114,-0.004342,-0.068793,12.352605,12.211148,-0.301843,-0.235158,-0.522246,12.226180
...,...,...,...,...,...,...,...,...,...
DTLRVILLA,-0.011751,0.050987,-0.068793,0.079192,-0.001122,-0.301843,-0.235158,-0.531121,-0.000678
boredpanda,-0.011751,0.050987,-0.068793,0.587950,-0.008026,-0.301843,-0.235158,-0.528903,-0.003918
tomecurran,-0.067114,-0.004342,-0.068793,0.734002,-0.012679,-0.301843,-0.235158,2.144640,-0.006877


# Save Results

In [31]:
results.to_csv(r'.\data\results.csv')

In [32]:
top_non_tweet_df = non_tweet_df.drop(columns=['source_id', 'target_id', 'type_of_content'])
top_non_tweet_df

,source,target
0,mrsmcpuffin,FAMU_MBB
1,mrsmcpuffin,KingJames
2,somenailstrips,WandaCo87259801
3,somenailstrips,Poshmarkapp
4,ja_corey14,originelllly
...,...,...
6004,products_hot,KaysUniverse11
6005,products_hot,Poshmarkapp
6008,Gluxia1,KingSize199619
6009,rgikpttn,Poshmarkapp


In [33]:
top_non_tweet_df.to_csv(r'.\data\edges.csv')